# Localization with time of arrival measurements

<a href="https://colab.research.google.com/github/oouabi/gtsam-examples/blob/main/TimeOfArrivalExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Acoustic sensors can be used as a mean to track a moving object. Typically, acoustic measurements are characteristic of the distance between the receiver and an acoustic source. Hence, they convey information on the position of the latter, which can be leveraged for localization. In the next, we will see how gtsam can be used in such a scenario. The following example considers an omnidirectional sound source that periodically emits an acoustic signal while moving. The signals are received by a fixed array of microphones that can synchronously record the signals from which Time of Arrival (ToA) measurements are extracted. The objective is to recover, from the ToA measurements, the trajectory of the moving agent. In this simple example, no odometry information is used, but odometry factors can seamlessly be integrated.

In [1]:
# Install the pre-requisites
%pip -q install gtbook 

<class 'AttributeError'>: module 'pexpect' has no attribute 'TIMEOUT'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
try:
    import google.colab
except:
    import plotly.io as pio
    pio.renderers.default = "png"

import gtsam
from gtbook.display import show
import gtsam.utils.plot as gtsam_plot

from gtsam import (LevenbergMarquardtOptimizer, LevenbergMarquardtParams,
                   NonlinearFactorGraph, Point3, Values, noiseModel)
from gtsam_unstable import Event, TimeOfArrival, TOAFactor

<class 'ModuleNotFoundError'>: No module named 'plotly'

# Example trajectory

We start by defining the microphones position and create the ground truth trajectory along with sound events. We first define useful units.

In [4]:
MS = 1e-3 # 1 millisecond in s
CM = 1e-2 # 1 centimeter in m

 Next, we istantiate a functor with a value $v = 330 m.s^{-1}$ for the speed of sound.

In [5]:
TIME_OF_ARRIVAL = TimeOfArrival(330)

<class 'NameError'>: name 'TimeOfArrival' is not defined

We define the positions of four microphones that we constrain to lie in a horizontal plane (*i.e* with constant $z$ coordinate).

In [6]:
height = 0.5
microphones = []
microphones.append(Point3(0, 0, height))
microphones.append(Point3(403 * CM, 0, height))
microphones.append(Point3(403 * CM, 403 * CM, height))
microphones.append(Point3(0, 403 * CM, 2 * height))

K = len(microphones)
for i in range(K):
    print("mic {} = {}".format(i, microphones[i]))

<class 'NameError'>: name 'Point3' is not defined

We subsequently create a ground truth trajectory for the moving sound emitter which follows a straight line, and specify the times at which a sound is emitted.

In [7]:
n = 5
groundTruth = []
timeOfEvent = 10

# simulate emitting a sound every second while moving on straight line
for key in range(n):
    groundTruth.append(
        Event(timeOfEvent, 245 * CM + key * 1.0, 201.5 * CM, (212 - 45) * CM))
    timeOfEvent += 1
        
#for event in groundTruth:
#    event.print()

<class 'NameError'>: name 'Event' is not defined

For visualization, we display the agent positions corresponding to an excitation event, along with the microphone array.

In [ ]:
fig = plt.figure()
plt.plot([mic[0] for mic in microphones], [mic[1] for mic in microphones], "b.", markersize=10, label="Microphones")
plt.plot([ev.location()[0] for ev in groundTruth], [ev.location()[1] for ev in groundTruth], "s", color="r", markersize=5, linestyle="-", lw=0.5, label="Source positions")
plt.xlabel("x [m]"); plt.ylabel("y [m]")
plt.legend(); fig.show()

We can now simulate the time of arrival measurements based on the distance between each microphone and the moving sound source. To determine the (noise-free) ToA between microphone $j$ for the $i$-th source position, one may simply use the formula:

$$ z_{i, j} = \frac{d_{i, j}}{v}, $$

where $d_{i, j}$ is the distance between them. Otherwise, the function TIME_OF_ARRIVAL.measure can take care of it.

In [8]:
def simulate_one_toa(microphones, event):
    """Simulate time-of-arrival measurements for a single event."""
    return [TIME_OF_ARRIVAL.measure(event, microphones[i])
            for i in range(len(microphones))]

simulatedTOA = [simulate_one_toa(microphones, event)
        for event in groundTruth]

for key in range(n):
    for i in range(K):
        print("z_{}{} = {} ms".format(key, i, simulatedTOA[key][i] / MS))

<class 'NameError'>: name 'K' is not defined

# Creation of the nonlinear factor graph

As always, we create a factor graph to solve the problem at hand. 

In [9]:
graph = NonlinearFactorGraph()

# Create a noise model for the TOA error
model = noiseModel.Isotropic.Sigma(1, 0.5 * MS)

K = len(microphones)
key = 0
for toa in simulatedTOA:
    for i in range(K):
        factor = TOAFactor(key, microphones[i], toa[i], model)
        graph.push_back(factor)
    key += 1
        
print(graph.at(0))

<class 'NameError'>: name 'NonlinearFactorGraph' is not defined

## Optimization and results

The optimizer needs to be provided an initial trajectory estimate.

In [ ]:
initial_estimate = Values()
zero = Event()
for key in range(n):
    TOAFactor.InsertEvent(key, zero, initial_estimate)
print(initial_estimate)

The graph is optimized using the Levenberg-Marquardt optimizer.

In [ ]:
params = LevenbergMarquardtParams()
params.setAbsoluteErrorTol(1e-10)
params.setVerbosityLM("SUMMARY")
optimizer = LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
print("Final Result:\n", result)

The visual representation of the result shows the excelent matching between estimated and ground truth positions.

In [ ]:
# parsing the result to retrieve the estimated locations
L = result.__repr__().split('\n'); L = [L[ind] for ind in range(2, len(L), 2)]
L = [l.split('location')[1].split()[1:] for l in L]
estimated_trajectory = [ [float(coord.split('}')[0]) for coord in l] for l in L]

# Display
fig = plt.figure()
plt.plot([mic[0] for mic in microphones], [mic[1] for mic in microphones], "b.", markersize=10, label="Microphones")
plt.plot([ev.location()[0] for ev in groundTruth], [ev.location()[1] for ev in groundTruth], "s", color="r", markersize=5, linestyle="-", lw=0.5, label="Source positions")
plt.plot([x[0] for x in estimated_trajectory], [x[1] for x in estimated_trajectory], "s", color="g", markersize=5, linestyle="-", lw=0.5, label="Estimated positions")
plt.xlabel("x [m]"); plt.ylabel("y [m]")
plt.legend(); fig.show()